In [67]:
import os

os.chdir("C:/Users/ehddl/Desktop/업무/code/sns-categorizer")

In [19]:
import boto3
from io import BytesIO
from dotenv import load_dotenv

import pandas as pd

load_dotenv()
aws_access_key = os.getenv("aws_accessKey")
aws_secret_key = os.getenv("aws_secretKey")
bucket_name = 'flexmatch-data'
region_name = 'ap-northeast-2'

s3 = boto3.client('s3',
                aws_access_key_id=aws_access_key,
                aws_secret_access_key=aws_secret_key,
                region_name=region_name)

today = datetime.now()
year, week, _ = today.isocalendar()

# profile_data_prefix = f'instagram-data/tables/EXTERNAL_2_RECENT_USER_INFO_MTR/year={year}/week={week}'
# media_data_prefix = f'instagram-data/tables/EXTERNAL_2_BY_USER_ID_MEDIA_DTL_INFO/year={year}/week={week}'

profile_data_prefix = f'instagram-data/tables/RECENT_USER_INFO_MTR/year={year}/week={week}/'
media_data_prefix = f'instagram-data/tables/BY_USER_ID_MEDIA_DTL_INFO/year={year}/week={week}/'

# profile 파일 목록
profile_response = s3.list_objects_v2(Bucket=bucket_name, Prefix=profile_data_prefix)
profile_keys = [obj['Key'] for obj in profile_response.get('Contents', []) if obj['Key'].endswith('.parquet')]

# media 파일 목록
media_response = s3.list_objects_v2(Bucket=bucket_name, Prefix=media_data_prefix)
media_keys = [obj['Key'] for obj in media_response.get('Contents', []) if obj['Key'].endswith('.parquet')]

profile_dfs = []

for key in profile_keys:
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    body = obj['Body'].read()
    df = pd.read_parquet(BytesIO(body))
    profile_dfs.append(df)

new_profile_data = pd.concat(profile_dfs, ignore_index=True)

media_dfs = []

for key in media_keys:
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    body = obj['Body'].read()
    df = pd.read_parquet(BytesIO(body))
    media_dfs.append(df)

new_media_data = pd.concat(media_dfs, ignore_index=True)

In [65]:
from DB_connection import *
from Model_inference import *

In [57]:
flexmatch_influencer_info = get_all_infos()

DB 접속 성공


In [58]:
# flexmatch_influencer_info['member_uid'] = flexmatch_influencer_info['member_uid'].astype('Int64')
# flexmatch_influencer_info['member_uid'] = flexmatch_influencer_info['member_uid'].fillna(0)
# flexmatch_influencer_info['member_uid'] = flexmatch_influencer_info['member_uid'].astype(int)
# flexmatch_influencer_info

flexmatch_influencer_info['member_uid'] = flexmatch_influencer_info['member_uid'].fillna(0).astype(int)
flexmatch_influencer_info

,user_id,member_uid,add1
0,admin,0,13F80C4C43DC63E772650995CA18639F
1,test901,5180,su.jin.1995
2,tkddmlwns^^^DD,0,oddity
3,ssh8918,0,https://www.instagram.com/flex.item/?hl=
4,yong,0,https://www.instagram.com/su.jin.1995/
...,...,...,...
1694,winter120@kakao.com,12474,dinyong0
1695,happyjyk@jr.naver.com,12476,juliejung60
1696,dong0987@gmail.com,12478,Lovely_rias
1697,lovelyn1122@naver.com,12487,kimrining


In [59]:
flexmatch_influencer_info['add1'] = flexmatch_influencer_info['add1'].str.replace('https://www.instagram.com/', '')

In [60]:
flexmatch_influencer_info['acnt_nm'] = flexmatch_influencer_info['add1'].str.replace('/', '')

In [ ]:
category_labels = ['IT', '게임', '결혼/연애', '교육', '다이어트/건강보조식품', '만화/애니/툰', '문구/완구', '미술/디자인', '반려동물', '베이비/키즈', '뷰티', '브랜드공식계정',
                    '사진/영상', '셀럽', '스포츠', '시사', '엔터테인먼트', '여행/관광', '유명장소/핫플', '일상', '자동차/모빌리티', '짤/밈', '취미', '패션', '푸드', '홈/리빙']

merged_df, predict_df = tokenize_and_predict_batch(new_profile_data, new_media_data, category_labels)
merged_df = merged_df[['acnt_id', 'acnt_nm']].reset_index(drop=True)
predict_df.reset_index(drop=True, inplace=True)

# final data after category labeling
final_predict_df = pd.concat([merged_df, predict_df], axis=1)

In [ ]:
db_merge_df = pd.merge(final_predict_df, flexmatch_influencer_info, on='acnt_nm', how='left')

main_category = db_merge_df.groupby(['acnt_id', 'acnt_nm'])['bert_top_label'].agg(lambda x: x.value_counts().idxmax()).to_frame().reset_index().rename(columns={'bert_top_label' : 'main_category'})

top_3_labels = db_merge_df.groupby(['acnt_id', 'acnt_nm'])['bert_top_label'].value_counts().groupby(level=[0,1]).head(3).reset_index(name='count')
top_3_labels_joined = (top_3_labels.groupby(['acnt_id', 'acnt_nm'])['bert_top_label'].apply(lambda x: '@'.join(x)).reset_index(name='top_3_category'))

top_3_labels_joined = top_3_labels_joined.drop(columns=['acnt_id', 'acnt_nm'])
final_df = pd.concat([main_category, top_3_labels_joined], axis=1)